In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from typing import Tuple, List, Dict, Any


In [2]:
df =  pd.read_csv("data/japan_n_skorea.csv")

In [3]:

def time_difference(time: pd.Series) -> pd.Series:
    """
    Calculate the time difference between the current time and the next time.
    
    :param time: A pandas Series of datetime values.
    :return: A pandas Series of time differences.
    """
    time_diff = time - time.shift(1)
    return time_diff

def process_time(time: pd.Series) -> pd.Series:
    """
    Process the time series data by converting to datetime, sorting, and calculating time differences.
    
    :param time: A pandas Series of datetime values.
    :return: A pandas Series of time differences in seconds.
    """
    time = pd.to_datetime(time)
    time = time.sort_values()
    dt = time_difference(time)
    dt = dt.dt.total_seconds()
    dt = dt[dt > 0].reset_index(drop=True)
    return dt


def analyze_distributions(in_series: pd.Series, significance: float = 0.05) -> Tuple[List[List[Any]], Dict[str, Dict[str, float]]]:
    """
    Analyze the distribution of the data and test for goodness of fit.
    
    :param in_series: A pandas Series of time differences.
    :param significance: Significance level for the goodness of fit tests.
    :return: A tuple containing the results of the analysis and the output parameters.
    """
    results = []
    output_parameters = {}

    # Uniform distribution
    fit_uniform_dist = stats.uniform(loc=(in_series.min()), scale=in_series.max() - in_series.min())
    test = stats.kstest(in_series, fit_uniform_dist.cdf)
    p_value = test[1]
    fit_status = "Good fit" if p_value > significance else "Bad fit"
    results.append(["Uniform Distribution", fit_status, p_value])
    output_parameters["Uniform Distribution"] = {
        "loc": in_series.min(),
        "scale": in_series.max() - in_series.min()
    }

    # Exponential distribution
    m1 = np.mean(in_series)
    fit_exponential_dist = stats.expon(scale=1 / m1)
    test = stats.kstest(in_series, fit_exponential_dist.cdf)
    p_value = test[1]
    fit_status = "Good fit" if p_value > significance else "Bad fit"
    results.append(["Exponential Distribution", fit_status, p_value])
    output_parameters["Exponential Distribution"] = {
        "estimated_lambda": 1 / m1
    }

    # Gamma distribution
    m2 = np.mean([x ** 2 for x in in_series])
    est_beta = m1 / (m2 - m1 ** 2)
    est_alpha = m1 * est_beta
    fit_gamma_dist = stats.gamma(a=est_alpha, scale=1 / est_beta)
    test = stats.kstest(in_series, fit_gamma_dist.cdf)
    p_value = test[1]
    fit_status = "Good fit" if p_value > significance else "Bad fit"
    results.append(["Gamma Distribution", fit_status, p_value])
    output_parameters["Gamma Distribution"] = {
        "estimated_alpha": est_alpha,
        "estimated_beta": est_beta
    }

    # Poisson distribution
    fi_poisson_dist = stats.poisson(mu=m1)
    test = stats.kstest(in_series, fi_poisson_dist.cdf)
    p_value = test[1]
    fit_status = "Good fit" if p_value > significance else "Bad fit"
    results.append(["Poisson Distribution", fit_status, p_value])
    output_parameters["Poisson Distribution"] = {
        "estimated_lambda": m1
    }

    # Normal distribution
    estimated_std = m2 - m1 ** 2
    fit_normal_dist = stats.norm(loc=m1, scale=estimated_std)
    test = stats.kstest(in_series, fit_normal_dist.cdf)
    p_value = test[1]
    fit_status = "Good fit" if p_value > significance else "Bad fit"
    results.append(["Normal Distribution", fit_status, np.format_float_scientific(test[1], precision=2)])
    output_parameters["Normal Distribution"] = {
        "estimated_mean": m1,
        "estimated_std": estimated_std
    }
    return results, output_parameters


def main(data:pd.DataFrame, case:str) -> Tuple[List[List[Any]], Dict[str, Dict[str, float]]]:
    """
    Main function to process time series data, analyze distributions, and plot the results.
    
    :return: A tuple containing the results of the analysis and the output parameters.
    """
    cases = ["All", "Large", "Small"]
    assert case in cases, f"Invalid case. Choose from {cases}"
    
    time = None
    
    if case == "All":
        time = data["time"]
    elif case == "Large":
        time = data[data["mag"] > 5]["time"]
    elif case == "Small":
        time = data[data["mag"] < 5]["time"]
        

    dt = process_time(time)
    results, output_parameters = analyze_distributions(dt)
    return results, output_parameters



In [4]:
results, output_parameters = main(df, "All")

In [5]:
def find_max_fit(results: List[List[Any]]) -> List[Any]:
    """
    Find the distribution with the maximum goodness of fit.
    
    :param results: A list containing the results of the analysis.
    :return: A list containing the distribution with the maximum goodness of fit.
    """
    max_fit = max([x[2] for x in results])
    return [x for x in results if x[2] == max_fit][0]


def generate_distribution(dist, output_parameters: Dict[str, Dict[str, float]], n) -> pd.DataFrame:
    """
    Create a pandas DataFrame containing the distribution parameters.
    
    :param dist: A string representing the distribution name.
    :param output_parameters: A dictionary containing the output parameters.
    :return: A pandas DataFrame containing the distribution parameters.
    """
    if dist == "Uniform Distribution":
        loc = output_parameters[dist]["loc"]
        scale = output_parameters[dist]["scale"]
        data = stats.uniform.rvs(loc=loc, scale=scale, size=n)
    elif dist == "Exponential Distribution":
        estimated_lambda = output_parameters[dist]["estimated_lambda"]
        data = stats.expon.rvs(scale=1 / estimated_lambda, size=n)
    elif dist == "Gamma Distribution":
        estimated_alpha = output_parameters[dist]["estimated_alpha"]
        estimated_beta = output_parameters[dist]["estimated_beta"]
        data = stats.gamma.rvs(a=estimated_alpha, scale=1 / estimated_beta, size=n)
    elif dist == "Poisson Distribution":
        estimated_lambda = output_parameters[dist]["estimated_lambda"]
        data = stats.poisson.rvs(mu=estimated_lambda, size=n)
    elif dist == "Normal Distribution":
        estimated_mean = output_parameters[dist]["estimated_mean"]
        estimated_std = output_parameters[dist]["estimated_std"]
        data = stats.norm.rvs(loc=estimated_mean, scale=estimated_std, size=n)
    return pd.DataFrame(data, columns=["data"])
    
    

In [6]:
output_parameters

{'Uniform Distribution': {'loc': np.float64(7.42),
  'scale': np.float64(3999488.52)},
 'Exponential Distribution': {'estimated_lambda': np.float64(5.917803647212268e-06)},
 'Gamma Distribution': {'estimated_alpha': np.float64(0.49280378256898094),
  'estimated_beta': np.float64(2.916316021846717e-06)},
 'Poisson Distribution': {'estimated_lambda': np.float64(168981.6120328824)},
 'Normal Distribution': {'estimated_mean': np.float64(168981.6120328824),
  'estimated_std': np.float64(57943518729.45413)}}